## Ideia basica para o projeto... apenas um rascunho com a interface

In [ ]:
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET

class Visualizador:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualizador de Objetos 2D")
        
        # Configurar o menu
        menu = tk.Menu(root)
        root.config(menu=menu)
        file_menu = tk.Menu(menu)
        menu.add_cascade(label="Arquivo", menu=file_menu)
        file_menu.add_command(label="Abrir", command=self.abrir_arquivo)
        
        # Frame principal para conter canvas e minimapa
        frame_principal = tk.Frame(root)
        frame_principal.pack(fill="both", expand=True)
        
        # Desenhar as viewports
        # estes dados deverão ser lidos do arquivo de entrada...
        largura = 800
        altura = 600
        # canvas da Viewport principal
        self.canvas = tk.Canvas(frame_principal, width=largura, height=altura, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)
        
        # Canvas do minimapa na lateral direita
        # usar um tamanho fixo para o minimapa
        self.minimap = tk.Canvas(frame_principal, width=150, height=120, bg="lightgrey") # razão de aspecto 4:3
        self.minimap.pack(side="right", padx=10, pady=10)


    # apenas abrir o arquivo e chamar a função para carregar seus dados...
    def abrir_arquivo(self):
        pass

    # ler os dados do arquivo aberto
    def carregar_arquivo(self, caminho):
        pass
    
    # aplicar a transformada de viewport a um ponto e retornar o ponto calculado
    def window2viewport(self, ponto, window, viewport):
        pass


    # redensenhar a viewport
    def desenhar_viewport(self):
        pass

    # desenhar a viewport do minimapa
    def desenhar_minimapa(self):
        pass
        
  
if __name__ == "__main__":
    root = tk.Tk()
    app = Visualizador(root)
    root.mainloop()
